# Practical Session 5: Deep Learning

In this practical, we will continue from where the lecture left off and learn more about using TensorFlow. 

The practical will cover a few different network architectures and we will look at different components that are often used in neural networks.

To start off, let's import TensorFlow into our notebook.

In [ ]:
import tensorflow as tf

Note: TensorFlow v2 was only released recently.  The v1 API is still available as a submodule, although we won't be using that in this notebook:

In [ ]:
tf.compat.v1

## Minimal TensorFlow Example

This is the first example from the lecture. We first create a network takes an input vector, multiplies it by a weight matrix, adds a weight vector, and returns the result.

`tf.Variable` defines model parameters, which can be trained (as we will see shortly).  Here, we initialise the matrix variable as a 3x3 matrix, with every entry as 1.  Meanwhile, we initialise the vector variable with every entry as 0.
`tf.linalg.matvec` multiplies a matrix and a vector.

In [ ]:
weight_matrix = tf.Variable(tf.ones(shape=(3,3)))
weight_vector = tf.Variable(tf.zeros(shape=(3,)))

def affine_transformation(input_vector):
    return tf.linalg.matvec(weight_matrix, input_vector) + weight_vector

result = affine_transformation([2.,3.,7.])
print(result)

## Training the Parameters

This is the second example from the lecture, showing how to optimise the parameters in your model.

We first define a network that takes an input vector, multiplies it with a matrix (defined above), and sums the elements of the resulting vector (using `tf.math.reduce_sum`).  We then define a loss function, as the square error.  Given a specific input and output, we can calculate the loss of applying the network to the input.

Next, we define an optimiser &ndash; here, we are using stochastic gradient descent (SGD) with learning rate 0.001.  We then use this optimiser to train this network for 10 epochs, over this single training point.  This optimises the output towards the target value 20.  Printing out the results, we can see that the output gradually moves towards the target.

In [ ]:
def network(input_vector):
    return tf.math.reduce_sum(affine_transformation(input_vector))

def loss_fn(predicted, gold):
    return tf.square(predicted - gold)

input = [2.,3.,7.]
gold_output = 20

def loss():
    return loss_fn(network(input), gold_output)

opt = tf.keras.optimizers.SGD(learning_rate=1e-3)

for epoch in range(10):
    opt.minimize(loss, var_list=[weight_matrix, weight_vector])
    print(network(input))

## Network Layers

For most cases, we don't actually need to create the trainable variables manually. Instead, the feedfoward layer is available as a pre-defined module.

We can define a network as a sequence of operations, using `tf.keras.Sequential`.  The first operation here is a dense feedforward layer (`tf.keras.layers.Dense`), which acts like the `affine_transfomation` function we defined earlier.  The second operation sums the elements of the vector &ndash; this isn't a standard operation, so we have used `tf.keras.layers.Lambda` to allow a user-defined function.

By default, the parameters in a layer (like `tf.keras.layers.Dense`) are initialised randomly.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_shape=(3,)),
    tf.keras.layers.Lambda(lambda x: tf.math.reduce_sum(x, axis=1))
])

Note that such a model expects the input data to be given as a *minibatch* &ndash; this means that the input tensor should have an extra index, which ranges over datapoints.  In our case, instead of passing a 3-dimensional input vector, we have to pass an Nx3 matrix, where N is the number of datapoints.  Here, we can apply the model to a single datapoint (a 1x3 matrix):

In [ ]:
model.predict(tf.constant([[2.,3.,7.]]))

Now that we have a model defined in terms of layers, let's replace the manually created variables of the previous section.

In [ ]:
def loss_fn(predicted, gold):
    return tf.square(predicted - gold)

input = tf.constant([[2.,3.,7.]])
gold_output = 20

def loss():
    return loss_fn(model(input), gold_output)

opt = tf.keras.optimizers.SGD(learning_rate=1e-3)

for epoch in range(10):
    opt.minimize(loss, var_list=model.trainable_variables)
    print(model(input))

In fact, for standard optimizers and loss functions, the TensorFlow API makes it even easier for us:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_shape=(3,)),
    tf.keras.layers.Lambda(lambda x: tf.math.reduce_sum(x))
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              loss='mean_squared_error')

input = tf.constant([[2.,3.,7.]])
gold_output = tf.constant([[20.]])

for epoch in range(10):
    model.train_on_batch(input, gold_output)
    print(model(input))

## Activation Functions

As we saw in the lecture, activation functions are what gives neural networks their power to model non-linear patterns in the data.  After applying an affine transformation, we then apply a non-linear activation function to each element.

There are a number of different activation functions to choose from.

The [**sigmoid** function](https://en.wikipedia.org/wiki/Logistic_function), also known as the logistic function, transforms the value into the range between 0 and 1.

In [ ]:
tf.keras.layers.Dense(100, activation='sigmoid')

The [**tanh** function](https://en.wikipedia.org/wiki/Hyperbolic_function) has a similar shape to the sigmoid function, but transforms the value into the range between -1 and 1.

In [ ]:
tf.keras.layers.Dense(100, activation='tanh')

The <a href="https://en.wikipedia.org/wiki/Rectifier_(neural_networks)">**Rectified Linear Unit** function</a>, or ReLU, is the identity for positive values, but maps all negative values to 0.

In [ ]:
tf.keras.layers.Dense(100, activation='relu')

For classification tasks, an important activation function is the [**softmax**](https://en.wikipedia.org/wiki/Softmax_function).  This is unlike the activation functions mentioned above, because it isn't applied to each element separately.  It converts a vector of scores into a probability distribution &ndash; after applying the softmax, all values are between 0 and 1, and together they sum to 1.  Higher scores are assigned to higher probabilities, via the formula:

$P(i) \propto \exp(x_i)$

Or, more explicitly (notice how the value of the denominator depends on all other values):

$P(i) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$

The softmax is often used in the output layer of a network performing classification, in order to predict a probability distribution over all the possible classes.  For example, the following model takes a 20-dimensional input, maps it to a 50-dimensional hidden layer, then maps that to a distribution over 10 output classes.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, input_shape=(20,), activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

## Operations and Useful Functions

TensorFlow has corresponding versions of all the main operations you might want to use. This means you can add them into your computation graph and into your neural network.  The most common operations are available in `tf`, and further operations are available in `tf.math`.

In [ ]:
tf.abs # absolute value
tf.negative # computes the negative value
tf.sign # returns 1, 0 or -1 depending on the sign of the input
tf.math.reciprocal # reciprocal 1/x
tf.square # return input squared
tf.round # return rounded value
tf.sqrt # square root
tf.math.rsqrt # reciprocal of square root
tf.pow # power
tf.exp # exponential

These operations can be applied to scalar values, but also to vectors, matrices and higher-order tensors. In the latter case, they will be applied element-wise. For example:

In [ ]:
print(tf.negative([3.2,-2.7]))
print(tf.square([1.5,-2.1]))

Some useful operations are performed over a whole vector/matrix tensor and return a single value (we saw `tf.reduce_sum` earlier):

In [ ]:
tf.reduce_sum # Add elements together
tf.reduce_mean # Average over elements
tf.reduce_min # Minimum value
tf.reduce_max # Maximum value
tf.argmax # Index of the largest value
tf.argmin # Index of the smallest value

## Adaptive Learning Rates

Above, we used stochastic gradient descent (SGD) to train our model.  This uses a fixed learning rate to update the parameters.  Several optimisation algorithms are based on SGD, but adaptively adjust the learning rate (usually for each parameter separately).

Different adaptive learning rate strategies are also implemented in TensorFlow as functions. For example:

In [ ]:
tf.keras.optimizers.SGD
tf.keras.optimizers.Adadelta
tf.keras.optimizers.Adam
tf.keras.optimizers.RMSprop

If you are interested in the differences between these strategies, [this blog post](http://ruder.io/optimizing-gradient-descent/) provides more details.

## Training an XOR Function

[XOR](https://en.wikipedia.org/wiki/XOR_gate) is the function that takes two binary values and returns 1 if one of them is 1 and the other 0, while returning 0 if both of them have the same value.

It can be a difficult function to learn and cannot be modelled with a linear model. But let's try anyway.

Our dataset consists of all the possible different states that XOR can take:

In [ ]:
xor_input = tf.constant([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
xor_output = tf.constant([0.0, 1.0, 1.0, 0.0])

Now we construct a linear network and optimize it on this dataset, printing out the predictions at each epoch:

In [ ]:
linear_model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(2,))])

linear_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
                     loss='mean_squared_error')

for epoch in range(50):
    linear_model.train_on_batch(xor_input, xor_output)
    if (epoch + 1) % 10 == 0:
        print('after {} epochs:'.format(epoch+1), linear_model(xor_input).numpy().reshape((4,)))

As you can see, it's not doing very well. Ideally, the predictions should be \[0, 1, 1, 0\], but in this case they are hovering around 0.5 for every input case.

In order to improve this architecture, let's add some non-linear layers into our model.

In [ ]:
nonlinear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(2,), activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

nonlinear_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1),
                        loss='mean_squared_error')

for epoch in range(50):
    nonlinear_model.train_on_batch(xor_input, xor_output)
    if (epoch + 1) % 10 == 0:
        print('after {} epochs:'.format(epoch+1), nonlinear_model(xor_input).numpy().reshape((4,)))

This is much better. The values are much closer to \[0, 1, 1, 0\] than before, and they will continue improving if we train for longer.  (But remember that the model is initialised randomly &ndash; if you run it a few times, you will see that the results vary with each run.)

Notice that we have used a higher learning rate for this network. It will still learn with a smaller learning rate, but will converge more slowly. As discussed in the lecture, the learning rate is a hyperparameter that can vary quite a bit depending on the network architecture and dataset.

## XOR Classification

We can also do classification with TensorFlow. For this, we often use the softmax activation function described above, which predicts the probability for each of the possible classes.

We also have to change the loss function, as squared error is not suitable for classification.  A suitable loss function is [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy).  Because the correct output has probability 1 for the correct class, and probability 0 for the rest, cross entropy is the same as the negative log probability of the correct class.  In other words, by minimising cross entropy, we are trying to find the maximum likelihood model.

We can change the XOR example above to perform classification instead.  In this case, we are constructing a binary classifier &ndash; choosing between the classes of 0 and 1.  The output now prints the predicted probabilities of the two classes.

In [ ]:
nonlinear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(2,), activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

nonlinear_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1),
                        loss='sparse_categorical_crossentropy')

for epoch in range(50):
    nonlinear_model.train_on_batch(xor_input, xor_output)
    if (epoch + 1) % 10 == 0:
        print('after {} epochs:'.format(epoch+1), nonlinear_model(xor_input).numpy(), sep='\n')

## Minibatching

For the XOR data, there are only 4 datapoints.  However, with realistic datasets, it is inefficient to train on the whole dataset at once, because this will require a lot of computation in order to make a single update step.  Instead, we can train on a batch of data at a time.  For example, taking batches of 2 datapoints for the XOR data:

In [ ]:
nonlinear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(2,), activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

nonlinear_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1),
                        loss='sparse_categorical_crossentropy')

BATCH_SIZE = 2

for epoch in range(50):
    for i in range(0,len(xor_input),BATCH_SIZE):
        input_batch = xor_input[i:i+BATCH_SIZE]
        output_batch = xor_output[i:i+BATCH_SIZE]
        nonlinear_model.train_on_batch(input_batch, output_batch)
    if (epoch + 1) % 10 == 0:
        print('after {} epochs:'.format(epoch+1), nonlinear_model(xor_input).numpy(), sep='\n')

Again, this kind of functionality is built into TensorFlow.  The following code trains the model with the given batch size and number of epochs.

In [ ]:
nonlinear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(2,), activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

nonlinear_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1),
                        loss='sparse_categorical_crossentropy')

nonlinear_model.fit(xor_input, xor_output, batch_size=2, epochs=50)

print('final loss:', nonlinear_model.evaluate(xor_input, xor_output))
print('final predictions:', nonlinear_model.predict(xor_input), sep='\n')

# Assignment: Classification of House Locations

In the first practical, you used the California House Prices Dataset in order to predict the prices of the houses based on various properties about the houses. In this assignment, we will experiment with TensorFlow and train a model to predict the "ocean proximity" of a house.

First, we read in the dataset:

In [ ]:
import pandas as pd
data = pd.read_csv('../DSPNP_practical1/housing/housing.csv')
data.head()

Next, we split the ocean proximity column from the other features and convert the values to numerical IDs. Remember, the ocean_proximity column already contains discrete classes, so it is well-suited for the classification task. However, these are strings and in order to optimize the softmax function in TensorFlow, we need numerical IDs instead of strings. We can use the pandas map function to do the conversion:

In [ ]:
X = data.copy().drop(["ocean_proximity"], axis=1)
Y = data.copy()["ocean_proximity"]
Y = data.copy()["ocean_proximity"].map({"<1H OCEAN":0, "INLAND":1, "ISLAND": 2, "NEAR BAY": 3, "NEAR OCEAN": 4}).values

Now, let's split off some data for development and testing:

In [ ]:
import sklearn as sk
import sklearn.model_selection, sklearn.impute

X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, train_size=0.8, random_state=1)
X_train, X_dev, Y_train, Y_dev = sk.model_selection.train_test_split(X_train, Y_train, test_size=0.2, train_size=0.8, random_state=1)

And finally, let's preprocess the input features before giving them to the network. We need to fill in missing values with the imputer, and standardize the values to a similar range using the scaler:

In [ ]:
imputer = sk.impute.SimpleImputer(strategy="median")
imputer.fit(X_train)

X_train = imputer.transform(X_train)
X_dev = imputer.transform(X_dev)
X_test = imputer.transform(X_test)

scaler = sk.preprocessing.StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

We now have a dataset that we can work with. 

Input features:

In [ ]:
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)
print(X_train[:3])

And the correstponding gold-standard labels:

In [ ]:
print(Y_train.shape)
print(Y_dev.shape)
print(Y_test.shape)
print(Y_train[:10])

Based on the code examples above, construct a TensorFlow model, then train, tune and test it on this dataset. Experiment with different model settings and hyperparameters. Calculate and evaluate classification accuracy - the percentage of datapoints where the predicted class matches the gold-standard class.

During the practical session, give examples of what you tried and what were your findings.

Some suggestions and tips:
 * The XOR classification code can be a good place to start.
 * The output layer needs to have size 5, because the dataset has 5 possible classes.
 * Try testing on the development set as you are training, to make sure you don't overfit.
 * Evaluate on the dev set as much as you want, but evaluate on the test set only after you have chosen a good set of hyperparameters.
 * You could try different learning rates, hidden layer sizes, learning strategies, etc.
 * Adaptive learning rates can (and sometimes should) be used together with a regular hand-picked learning rate, and different adaptive learning rates can prefer very different regular learning rates.